# 🧱 DCGAN - Bricks Data

In [1]:
from dcgan import DCGANGenerator

import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import torch

2024-10-23 15:43:18.815066: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 15:43:19.452431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

# Clase personalizada para cargar imágenes desde carpetas
class ViolenceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.load_images()

    def load_images(self):
        violence_dir = os.path.join(self.root_dir, 'violencia')
        no_violence_dir = os.path.join(self.root_dir, 'no_violencia')

        for img_file in os.listdir(violence_dir):
            self.image_paths.append(os.path.join(violence_dir, img_file))
            self.labels.append(1)  # Etiqueta para violencia

        for img_file in os.listdir(no_violence_dir):
            self.image_paths.append(os.path.join(no_violence_dir, img_file))
            self.labels.append(0)  # Etiqueta para no violencia

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

In [ ]:
bases=["ubi","rlvsd"]
porcentajes=["1","5","10","25","50","75",]





In [4]:
if __name__ == "__main__":
    torch.cuda.empty_cache()
    print("> Iniciando proceso de Generacion de  muestras con DCGAN")
    for base in bases:
        for i in porcentajes:
            print(">>> Base de Datos: ",base," al ",i,"% de balance")
            rutaDeImagenesDesbalanceadas="./Datasets/Desbalanceadas/"+base+"/Database("+i+")/train/"
            rutaDeImagenesSinteticas="./Datasets/ImagenesSinteticas/"+base+"/Database("+i+")/"
            print("Ruta de imagenes desbalanceadas: "+rutaDeImagenesDesbalanceadas)


            data_dir =  rutaDeImagenesDesbalanceadas #
            batch_size = 8

            # Transformaciones para las imágenes
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])

            # Cargar dataset de entrenamiento
            dataset = ViolenceDataset(root_dir=data_dir, transform=transform)
            #train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

            # Determinar cuántas muestras faltan en la clase "violencia"
            num_violencia = sum([1 for _, label in dataset if label == 1])
            num_no_violencia = sum([1 for _, label in dataset if label == 0])
            samples_to_generate = num_no_violencia - num_violencia
            


            #samples_to_generate=1



            print(f'Muestras actuales de "violencia": {num_violencia}')
            print(f'Muestras actuales de "no violencia": {num_no_violencia}')
            print(f'Muestras a generar: {samples_to_generate}')



            # Filtrar el dataset para obtener solo las imágenes de "violencia"
            violence_images = [image for image, label in dataset if label == 1]
            violence_images_tensor = torch.stack(violence_images)

            print("Ruta  de imagenes sinteticas: "+rutaDeImagenesSinteticas)
            # Definir el directorio donde guardar las imágenes
            save_dir = rutaDeImagenesSinteticas  # Cambia esto por el path de tu carpeta de salida


            gan=DCGANGenerator()
            gan.generateImages(numSamples=samples_to_generate,imageInputPath=rutaDeImagenesDesbalanceadas,imageOutputPath=save_dir,epocas=300,lote=64)

            print(f"{samples_to_generate} imágenes sintéticas guardadas en {save_dir}")

            print("··············································································")
            print()
            print()

> Iniciando proceso de Generacion de  muestras con DCGAN
>>> Base de Datos:  ubi  al  50 % de balance
Ruta de imagenes desbalanceadas: ./Datasets/Desbalanceadas/ubi/Database(50)/train/
Muestras actuales de "violencia": 5250
Muestras actuales de "no violencia": 10500
Muestras a generar: 5250
Ruta  de imagenes sinteticas: ./Datasets/ImagenesSinteticas/ubi/Database(50)/
Found 5250 files belonging to 1 classes.


2024-10-23 15:43:57.203772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 15:43:57.233894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 15:43:57.234086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        3072      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 128)      512       
 ormalization)                                                   
                                                             

2024-10-23 15:43:57.814664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5250]
	 [[{{node Placeholder/_0}}]]
2024-10-23 15:43:57.814876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5250]
	 [[{{node Placeholder/_0}}]]
2024-10-23 15:43:59.922222: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-23 15:44:00.194256: I tensorflow/compiler/x

83/83 [==============================] - 8s 46ms/step - d_loss: -0.0551 - d_real_acc: 0.9082 - d_fake_acc: 0.9227 - d_acc: 0.9154 - g_loss: 7.2704 - g_acc: 0.0773
Epoch 2/300
83/83 [==============================] - 3s 37ms/step - d_loss: -0.1641 - d_real_acc: 0.9577 - d_fake_acc: 0.9657 - d_acc: 0.9617 - g_loss: 5.5270 - g_acc: 0.0343
Epoch 3/300
83/83 [==============================] - 3s 37ms/step - d_loss: -0.0016 - d_real_acc: 0.9297 - d_fake_acc: 0.9608 - d_acc: 0.9452 - g_loss: 6.8993 - g_acc: 0.0392
Epoch 4/300
83/83 [==============================] - 3s 36ms/step - d_loss: 0.1936 - d_real_acc: 0.8640 - d_fake_acc: 0.8693 - d_acc: 0.8667 - g_loss: 3.2376 - g_acc: 0.1307
Epoch 5/300
83/83 [==============================] - 3s 37ms/step - d_loss: 0.1728 - d_real_acc: 0.8613 - d_fake_acc: 0.8768 - d_acc: 0.8690 - g_loss: 3.1831 - g_acc: 0.1232
Epoch 6/300
83/83 [==============================] - 3s 37ms/step - d_loss: 0.4167 - d_real_acc: 0.8194 - d_fake_acc: 0.8097 - d_acc: 0.814

...

INFO:tensorflow:Assets written to: ./models/generator/assets


2024-10-23 15:59:30.871108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,64]
	 [[{{node inputs}}]]
2024-10-23 15:59:30.884731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,16,16,128]
	 [[{{node inputs}}]]
2024-10-23 15:59:30.896391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,8,8,256]
	 [[{{node inputs}}]]
2

INFO:tensorflow:Assets written to: ./models/discriminator/assets


INFO:tensorflow:Assets written to: ./models/discriminator/assets


165/165 [==============================] - 1s 3ms/step
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/eO3qEpduzJ6XEqTs.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/Gwx5if401kL9ZhCk.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/1Ye3ukAeIjuh2TyB.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/2PEnTa5jv1uXvgF1.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/b22Hw98m4q2kExDo.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/4YxsuxCYUf8TnLlq.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/mCu0gSzMYPjojrbS.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/eTthp0cbLLltqOGt.jpg  generada sinteticamente
>>> Imagen:  ./Datasets/ImagenesSinteticas/ubi/Database(50)/2URIAouRKMQMbqmw.jpg  generada sintet